# ビジネス関連法案の RAG 検索 (e-GOV 法令検索)


In [13]:
import logging
import cohere
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from typing import List, Dict
import time
import re
from dotenv import load_dotenv
import os

In [2]:
load_dotenv()
co = cohere.ClientV2(os.environ['COHERE_APIKEY'])

EGOV_SEARCH_URL = "https://elaws.e-gov.go.jp/api/1/lawlists/1"

/Users/aki/tech/ai-api-playground/env/lib/python3.14/site-packages/cohere/core/pydantic_utilities.py:13: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.datetime_parse import parse_date as parse_date


# 2. e-Gov から法令情報を取得

In [3]:

def fetch_egov_laws(keyword: str = "", limit: int = 50) -> List[Dict]:
    """
    e-Gov APIから法令情報を取得
    
    Args:
        keyword: 検索キーワード
        limit: 取得する法令の最大数
    
    Returns:
        法令情報のリスト
    """
    laws = []
    
    try:
        # e-Gov API にリクエスト
        response = requests.get(EGOV_SEARCH_URL, timeout=30)
        response.raise_for_status()
        
        # XMLパース
        soup = BeautifulSoup(response.content, 'xml')
        law_items = soup.find_all('LawNameListInfo')
        
        for item in law_items[:limit]:
            law_num = item.find('LawNo')
            law_name = item.find('LawName')
            
            if law_num and law_name:
                laws.append({
                    'law_number': law_num.text.strip(),
                    'law_name': law_name.text.strip(),
                    'text': f"{law_name.text.strip()} ({law_num.text.strip()})"
                })
        
        print(f"✓ {len(laws)}件の法令情報を取得しました")
        
    except Exception as e:
        print(f"⚠ エラー: {e}")
        # サンプルデータをフォールバック
        laws = get_sample_laws()
    
    return laws

import json
def get_sample_laws() -> List[Dict]:
    """サンプル法令データ（API接続失敗時のフォールバック）"""
    with open('data/egov-sample.json') as f:
        return json.load(f)

def save_as_sample_json(laws):
    with open('data/egov-sample.json', "w") as f:
        json.dump(laws, f, indent=2)

print("法令情報を取得中...")
laws = fetch_egov_laws(limit=100)



法令情報を取得中...
✓ 100件の法令情報を取得しました


In [6]:
# データフレームに変換
df_laws = pd.DataFrame(laws)
print(f"\n取得した法令数: {len(df_laws)}")
df_laws.head()


取得した法令数: 100


,law_number,law_name,text
0,昭和二十一年憲法,日本国憲法,日本国憲法 (昭和二十一年憲法)
1,明治六年太政官布告第六十五号,明治六年太政官布告第六十五号（絞罪器械図式）,明治六年太政官布告第六十五号（絞罪器械図式） (明治六年太政官布告第六十五号)
2,明治十七年太政官布告第三十二号,明治十七年太政官布告第三十二号（爆発物取締罰則）,明治十七年太政官布告第三十二号（爆発物取締罰則） (明治十七年太政官布告第三十二号)
3,明治二十二年法律第三十四号,明治二十二年法律第三十四号（決闘罪ニ関スル件）,明治二十二年法律第三十四号（決闘罪ニ関スル件） (明治二十二年法律第三十四号)
4,明治二十三年法律第一号,保管金規則,保管金規則 (明治二十三年法律第一号)


In [19]:
def create_embeddings(texts: List[str], batch_size: int = 96) -> np.ndarray:
    """
    テキストリストをCohere APIでベクトル化
    
    Args:
        texts: ベクトル化するテキストのリスト
        batch_size: バッチサイズ
    
    Returns:
        埋め込みベクトルの配列
    """
    embeddings = []
    
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]
        try:
            response = co.embed(
                texts=batch,
                model='embed-multilingual-v3.0',
                input_type='search_document'
            )
            #logging.warning(response.embeddings)
            embeddings.extend(response.embeddings)
            print(f"✓ {i+len(batch)}/{len(texts)} 件処理完了")
            time.sleep(0.5)  # レート制限対策
        except Exception as e:
            print(f"⚠ エラー (batch {i}): {e}")
            break
    return np.array(embeddings, dtype=object)

# 法令名をベクトル化
print("\n法令情報をベクトル化中...")
law_texts = df_laws['text'].tolist()
law_embeddings = create_embeddings(law_texts)

print(f"✓ ベクトル化完了: {law_embeddings.shape}")


法令情報をベクトル化中...
✓ 96/100 件処理完了
✓ 100/100 件処理完了
✓ ベクトル化完了: (12, 2)


In [29]:
# =============================================================================
# 4. RAG検索関数の実装
# =============================================================================

def semantic_search(query: str, top_k: int = 10) -> pd.DataFrame:
    """
    セマンティック検索を実行
    
    Args:
        query: 検索クエリ
        top_k: 上位何件を返すか
    
    Returns:
        検索結果のデータフレーム
    """
    # クエリをベクトル化
    query_response = co.embed(
        texts=[query],
        model='embed-multilingual-v3.0',
        input_type='search_query'
    )
    query_embedding = np.array(query_response.embeddings["float"][0])
    
    # コサイン類似度を計算
    similarities = np.dot(law_embeddings, query_embedding) / (
        np.linalg.norm(law_embeddings, axis=1) * np.linalg.norm(query_embedding)
    )
    
    # 上位k件を取得
    top_indices = np.argsort(similarities)[::-1][:top_k]
    
    results = df_laws.iloc[top_indices].copy()
    results['similarity_score'] = similarities[top_indices]
    
    return results

def rerank_results(query: str, results: pd.DataFrame, top_k: int = 5) -> pd.DataFrame:
    """
    Cohere Rerank APIで結果を再ランキング
    
    Args:
        query: 検索クエリ
        results: 初期検索結果
        top_k: 最終的に返す件数
    
    Returns:
        再ランキング後の結果
    """
    documents = results['text'].tolist()
    
    try:
        rerank_response = co.rerank(
            query=query,
            documents=documents,
            model='rerank-multilingual-v3.0',
            top_n=top_k
        )
        
        # 再ランキング結果を整理
        reranked_indices = [r.index for r in rerank_response.results]
        reranked_scores = [r.relevance_score for r in rerank_response.results]
        
        reranked_results = results.iloc[reranked_indices].copy()
        reranked_results['rerank_score'] = reranked_scores
        
        return reranked_results
        
    except Exception as e:
        print(f"⚠ Rerank エラー: {e}")
        return results.head(top_k)


# サンプル実行

In [30]:
business_queries = [
    "会社設立や株式会社の運営に関する法律",
    "従業員の雇用や労働条件に関する法律",
    "個人情報やデータ保護に関する法律",
    "契約や取引に関する法律",
    "知的財産権に関する法律"
]

all_business_laws = []

for query in business_queries:
    print(f"\n🔍 検索: {query}")
    print("-" * 70)
    
    # セマンティック検索
    search_results = semantic_search(query, top_k=10)
    """
    # Rerankで精度向上
    final_results = rerank_results(query, search_results, top_k=3)
    
    # 結果を表示
    for idx, row in final_results.iterrows():
        print(f"  • {row['law_name']}")
        print(f"    法令番号: {row['law_number']}")
        print(f"    関連度: {row.get('rerank_score', row.get('similarity_score', 0)):.3f}")
        print()
        
        all_business_laws.append({
            'query': query,
            'law_name': row['law_name'],
            'law_number': row['law_number'],
            'score': row.get('rerank_score', row.get('similarity_score', 0))
        })
    """



🔍 検索: 会社設立や株式会社の運営に関する法律
----------------------------------------------------------------------


TypeError: 'EmbedByTypeResponseEmbeddings' object is not subscriptable